In [1]:
import os
import json
import requests
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [4]:
model = "gemini-pro"
prompt = "北京下大雪了，以李白的风格写一首诗：\n"
url = f"https://googleapis.fmr.wiki/v1beta/models/{model}:generateContent?key={os.getenv('GOOGLE_API_KEY')}"
headers = {"Content-Type": "application/json"}
data = {
    "contents": [{
        "parts":[{
            "text": prompt
        }]
    }]
}

In [5]:
ret = requests.post(url, json.dumps(data), headers=headers)

In [16]:
text = ret.json()['candidates'][0]['content']['parts'][0]['text']

In [17]:
print(text)

**雪落长安**

飞雪漫天舞，
长安银装素裹。
千门万户闭，
万籁俱寂。

片片雪花飄，
如柳絮随风舞。
簌簌花枝落，
如天女散花。

雪落屋檐下，
如晶莹的珍珠。
雪落树梢头，
如绽放的梨花。

雪落大地，
如铺上了白色的地毯。
雪落山川，
如披上了银色的纱衣。

雪，如诗，
如画，
如梦，
如幻。

雪，是冬天的使者，
带来寒冷，也带来寂静。
雪，是自然的精灵，
带来美丽，也带来希望。
